[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp24/blob/main/4.topics/TopicModel.ipynb)

In this notebook we'll explore topic modeling to discover broad themes in a collection of movie summaries.

In [ ]:
import nltk
import re
import gensim
from gensim import corpora
import operator

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

import numpy as np
import random

random.seed(1)

In [ ]:
!wget https://raw.githubusercontent.com/dbamman/anlp24/main/data/jockers.stopwords
!wget https://raw.githubusercontent.com/dbamman/anlp24/main/data/movie.metadata.tsv
!wget https://raw.githubusercontent.com/dbamman/anlp24/main/data/plot_summaries.txt

In [ ]:
def read_stopwords(filename):
    stopwords={}
    with open(filename) as file:
        for line in file:
            stopwords[line.rstrip()]=1
    return stopwords

Since we're running topic modeling on texts with lots of names, we'll add the Jockers list of stopwords (which includes character names) to our stoplist.

In [ ]:
stop_words = {k:1 for k in stopwords.words('english')}
stop_words.update(read_stopwords("jockers.stopwords"))
stop_words["'s"]=1
stop_words=list(stop_words.keys())

In [ ]:
def filter(word, stopwords):

    """ Function to exclude words from a text """

    # no stopwords
    if word in stopwords:
        return False

    # has to contain at least one letter
    if re.search("[A-Za-z]", word) is not None:
        return True

    return False

In [ ]:
def read_docs(plotFile, metadataFile, stopwords):

    names={}
    box={}

    with open(metadataFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            name=cols[2]
            boxoffice=cols[4]
            if len(boxoffice) != 0:
                box[idd]=int(boxoffice)
                names[idd]=name

    n=5000
    target_movies={}


    sorted_box = sorted(box.items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_box[:n]:
        target_movies[k]=names[k]

    docs=[]
    names=[]

    with open(plotFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            text=cols[1]

            if idd in target_movies:
                tokens=nltk.word_tokenize(text.lower())
                tokens=[x for x in tokens if filter(x, stopwords)]
                docs.append(tokens)
                name=target_movies[idd]
                names.append(name)
    return docs, names

We'll read in summaries of the 5,000 movies with the highest box office revenues.

In [ ]:
metadataFile="movie.metadata.tsv"
plotFile="plot_summaries.txt"
data, doc_names=read_docs(plotFile, metadataFile, stop_words)

We will convert the movie summaries into a bag-of-words representation using gensim's [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) methods.

In [ ]:
# Create vocab from data; restrict vocab to only the top 10K terms that show up in at least 5 documents
# and no more than 50% of all documents

dictionary = corpora.Dictionary(data)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)

In [ ]:
# Replace dataset with numeric ids words in vocab (and exclude all other words)
corpus = [dictionary.doc2bow(text) for text in data]

In [ ]:
num_topics=20

Now let's run a topic model on this data using gensim's built-in LDA.

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics,
                                           passes=10,
                                           alpha='auto')

We can get a sense of what the topics are by printing the top 10 words with highest $P(word \mid topic)$ for each topic

In [ ]:
for i in range(num_topics):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model.show_topic(i, topn=10)])))

Another way of understanding topics is to print out the documents that have the highest topic representation -- i.e., for a given topic $k$, the documents with highest $P(topic=k | document)$.  How much do the documents listed here align with your understanding of the topics?

In [ ]:
topic_model=lda_model

topic_docs=[]
for i in range(num_topics):
    topic_docs.append({})
for doc_id in range(len(corpus)):
    doc_topics=topic_model.get_document_topics(corpus[doc_id])
    for topic_num, topic_prob in doc_topics:
        topic_docs[topic_num][doc_id]=topic_prob

for i in range(num_topics):
    print("%s\n" % ' '.join([term for term, freq in topic_model.show_topic(i, topn=10)]))
    sorted_x = sorted(topic_docs[i].items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x[:5]:
        print("%s\t%.3f\t%s" % (i,v,doc_names[k]))
    print()